#### Start a simple Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()

#### Read the CSV

In [2]:
df = spark.read.csv('../data/original.csv', header=True, inferSchema=True)

#### If city is null, then set city to 'Unknown'

In [3]:
from pyspark.sql.functions import *

df_city = df.withColumn('City_N',when(df.City.isNull(),'Unknown').otherwise(df.City))
df_city.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|         City_N|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

### If the Job title is null, then delete the row

In [4]:
df_jt = df.filter(df.JobTitle.isNotNull())
df_jt.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36|45.1905186|  0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Pr

### If Salary is null then replace it with Average salary

In [12]:
df_sal = df.withColumn('Salary_fix', df.Salary.substr(2,100).cast('float'))
mean_salary = df_sal.groupBy().avg('Salary_fix').take(1)[0][0]
print(mean_salary)
df_sal = df_sal.withColumn('New_salary', when(df_sal.Salary_fix.isNull(),lit(mean_salary)).otherwise(df_sal.Salary_fix))

df_sal.show()

55487.95562890625
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+----------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|Salary_fix|      New_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+----------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|  57438.18|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|   62846.6|   62846.6015625|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52|39.9947462|116.3397725|  57576.52|  57576.51953125|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  61489.23|  61489.23046875|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi| 